In [1]:
import pandas as pd

In [2]:
df= pd.read_csv("Backpacks.csv")

In [3]:
df.head()

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
0,0,Jansport,Leather,Medium,7,1,0,Tote,Black,14,113
1,1,Jansport,Canvas,Small,10,1,1,Messenger,Green,28,69
2,2,Under Armour,Leather,Small,2,1,0,Messenger,Red,17,39
3,3,Nike,Nylon,Small,8,1,0,Messenger,Green,13,81
4,4,Adidas,Canvas,Medium,1,1,1,Messenger,Green,18,86


In [4]:
df.isnull().sum()

id                      0
Brand                   0
Material                0
Size                    0
Compartments            0
Laptop Compartment      0
Waterproof              0
Style                   0
Color                   0
Weight Capacity (kg)    0
Price                   0
dtype: int64

Univariate_Analysis

In [5]:
df = pd.get_dummies(df, columns=['Brand', 'Material', 'Style','Size','Color'], drop_first=True)

In [6]:
numerical=df.select_dtypes(include=['float64','int64']).columns
df_nums=df[numerical]

In [7]:
df['Price_per_capacity'] = df['Price'] / df['Weight Capacity (kg)']

In [8]:
df.head()

,id,Compartments,Laptop Compartment,Waterproof,Weight Capacity (kg),Price,Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,...,Style_Messenger,Style_Tote,Size_Medium,Size_Small,Color_Blue,Color_Gray,Color_Green,Color_Pink,Color_Red,Price_per_capacity
0,0,7,1,0,14,113,True,False,False,False,...,False,True,True,False,False,False,False,False,False,8.071429
1,1,10,1,1,28,69,True,False,False,False,...,True,False,False,True,False,False,True,False,False,2.464286
2,2,2,1,0,17,39,False,False,False,True,...,True,False,False,True,False,False,False,False,True,2.294118
3,3,8,1,0,13,81,False,True,False,False,...,True,False,False,True,False,False,True,False,False,6.230769
4,4,1,1,1,18,86,False,False,False,False,...,True,False,True,False,False,False,True,False,False,4.777778


In [9]:
from sklearn.model_selection import train_test_split

# Example: Splitting dataset
X = df.drop(columns=['Price'])  # Features
y = df['Price']  # Target variable

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
import xgboost as xgb

model = xgb.XGBRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [11]:
y_pred=model.predict(X_test)

In [12]:
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
print("r2",r2_score(y_test,y_pred))
print("mse",mean_squared_error(y_test,y_pred))
print("mae", mean_absolute_error(y_test,y_pred))

r2 0.9996794724666411
mse 0.4860658646361949
mae 0.32398205920855205


In [13]:
from sklearn.model_selection import cross_val_score

cv_r2 = cross_val_score(model, X, y, cv=5, scoring='r2')
print("Mean CV R²:", cv_r2.mean())

Mean CV R²: 0.9996470745754751


In [14]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
import numpy as np

# Define the parameter grid
param_dist = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

# Initialize the model
xgb = XGBRegressor(objective='reg:squarederror', random_state=42)

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=10,  # Number of random combinations to try
    scoring='r2',
    cv=3,
    n_jobs=-1,  # Use all CPU cores
    verbose=1
)

# Fit the random search model
random_search.fit(X_train, y_train)

# Print the best parameters
print("Best Hyperparameters:", random_search.best_params_)

# Evaluate the model
best_xgb = XGBRegressor(**random_search.best_params_)
best_xgb.fit(X_train, y_train)
y_pred = best_xgb.predict(X_test)

# Performance metrics
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
print("R² Score:", r2_score(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best Hyperparameters: {'subsample': 1.0, 'n_estimators': 300, 'max_depth': 7, 'learning_rate': 0.1, 'colsample_bytree': 1.0}
R² Score: 0.999688456949567
MSE: 0.47244129261922146
MAE: 0.3052989412466685


In [15]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
import numpy as np

# Initialize the model
xgb = XGBRegressor(
    n_estimators=200, 
    learning_rate=0.05, 
    max_depth=7, 
    subsample=1.0, 
    colsample_bytree=1.0, 
    random_state=42
)

# Cross-validation (3 folds as an example)
cv_scores = cross_val_score(xgb, X_train, y_train, cv=3, scoring='r2')  # R² score

# Print the cross-validation results
print("Cross-validation scores for each fold:", cv_scores)
print("Average cross-validation score:", np.mean(cv_scores))


Cross-validation scores for each fold: [0.99962584 0.99965058 0.99966077]
Average cross-validation score: 0.9996457331168317


In [16]:
df_test=pd.read_csv("Backpack_test1.csv")

In [17]:
df_test.head()

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Unnamed: 10,Unnamed: 11
0,300000,Puma,Leather,Small,2,0,0,Tote,Green,21,NaN,NaN
1,300001,Nike,Canvas,Medium,7,0,1,Backpack,Green,14,NaN,NaN
2,300002,Adidas,Canvas,Large,9,0,1,Messenger,Blue,12,NaN,NaN
3,300003,Adidas,Nylon,Large,1,1,0,Messenger,Green,18,NaN,NaN
4,300004,unknown,Nylon,Large,2,1,1,Tote,Black,10,NaN,NaN


In [18]:
X

,id,Compartments,Laptop Compartment,Waterproof,Weight Capacity (kg),Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,Brand_unknown,...,Style_Messenger,Style_Tote,Size_Medium,Size_Small,Color_Blue,Color_Gray,Color_Green,Color_Pink,Color_Red,Price_per_capacity
0,0,7,1,0,14,True,False,False,False,False,...,False,True,True,False,False,False,False,False,False,8.071429
1,1,10,1,1,28,True,False,False,False,False,...,True,False,False,True,False,False,True,False,False,2.464286
2,2,2,1,0,17,False,False,False,True,False,...,True,False,False,True,False,False,False,False,True,2.294118
3,3,8,1,0,13,False,True,False,False,False,...,True,False,False,True,False,False,True,False,False,6.230769
4,4,1,1,1,18,False,False,False,False,False,...,True,False,True,False,False,False,True,False,False,4.777778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,299995,9,0,0,13,False,False,False,False,False,...,False,True,False,True,True,False,False,False,False,10.000000
299996,299996,6,0,1,27,True,False,False,False,False,...,False,True,False,False,True,False,False,False,False,0.740741
299997,299997,9,1,1,12,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,9.250000
299998,299998,1,0,1,6,False,False,False,False,False,...,False,True,False,True,False,False,False,True,False,19.333333


In [19]:
df_test = pd.get_dummies(df_test, columns=['Brand', 'Material', 'Style','Size','Color'], drop_first=True)

In [20]:
df_test.head()

,id,Compartments,Laptop Compartment,Waterproof,Weight Capacity (kg),Unnamed: 10,Unnamed: 11,Brand_Jansport,Brand_Nike,Brand_Puma,...,Material_Polyester,Style_Messenger,Style_Tote,Size_Medium,Size_Small,Color_Blue,Color_Gray,Color_Green,Color_Pink,Color_Red
0,300000,2,0,0,21,NaN,NaN,False,False,True,...,False,False,True,False,True,False,False,True,False,False
1,300001,7,0,1,14,NaN,NaN,False,True,False,...,False,False,False,True,False,False,False,True,False,False
2,300002,9,0,1,12,NaN,NaN,False,False,False,...,False,True,False,False,False,True,False,False,False,False
3,300003,1,1,0,18,NaN,NaN,False,False,False,...,False,True,False,False,False,False,False,True,False,False
4,300004,2,1,1,10,NaN,NaN,False,False,False,...,False,False,True,False,False,False,False,False,False,False


In [21]:
df_test.head()

,id,Compartments,Laptop Compartment,Waterproof,Weight Capacity (kg),Unnamed: 10,Unnamed: 11,Brand_Jansport,Brand_Nike,Brand_Puma,...,Material_Polyester,Style_Messenger,Style_Tote,Size_Medium,Size_Small,Color_Blue,Color_Gray,Color_Green,Color_Pink,Color_Red
0,300000,2,0,0,21,NaN,NaN,False,False,True,...,False,False,True,False,True,False,False,True,False,False
1,300001,7,0,1,14,NaN,NaN,False,True,False,...,False,False,False,True,False,False,False,True,False,False
2,300002,9,0,1,12,NaN,NaN,False,False,False,...,False,True,False,False,False,True,False,False,False,False
3,300003,1,1,0,18,NaN,NaN,False,False,False,...,False,True,False,False,False,False,False,True,False,False
4,300004,2,1,1,10,NaN,NaN,False,False,False,...,False,False,True,False,False,False,False,False,False,False


In [22]:
df_test['Price_per_capacity']=df['Price_per_capacity']

In [23]:
df_test.isnull().sum()

id                           0
Compartments                 0
Laptop Compartment           0
Waterproof                   0
Weight Capacity (kg)         0
Unnamed: 10             200000
Unnamed: 11             199999
Brand_Jansport               0
Brand_Nike                   0
Brand_Puma                   0
Brand_Under Armour           0
Brand_unknown                0
Material_Leather             0
Material_Nylon               0
Material_Polyester           0
Style_Messenger              0
Style_Tote                   0
Size_Medium                  0
Size_Small                   0
Color_Blue                   0
Color_Gray                   0
Color_Green                  0
Color_Pink                   0
Color_Red                    0
Price_per_capacity           0
dtype: int64

In [24]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 23 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    300000 non-null  int64  
 1   Compartments          300000 non-null  int64  
 2   Laptop Compartment    300000 non-null  int64  
 3   Waterproof            300000 non-null  int64  
 4   Weight Capacity (kg)  300000 non-null  int64  
 5   Brand_Jansport        300000 non-null  bool   
 6   Brand_Nike            300000 non-null  bool   
 7   Brand_Puma            300000 non-null  bool   
 8   Brand_Under Armour    300000 non-null  bool   
 9   Brand_unknown         300000 non-null  bool   
 10  Material_Leather      300000 non-null  bool   
 11  Material_Nylon        300000 non-null  bool   
 12  Material_Polyester    300000 non-null  bool   
 13  Style_Messenger       300000 non-null  bool   
 14  Style_Tote            300000 non-null  bool   
 15  

In [25]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 25 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    200000 non-null  int64  
 1   Compartments          200000 non-null  int64  
 2   Laptop Compartment    200000 non-null  int64  
 3   Waterproof            200000 non-null  int64  
 4   Weight Capacity (kg)  200000 non-null  int64  
 5   Unnamed: 10           0 non-null       float64
 6   Unnamed: 11           1 non-null       float64
 7   Brand_Jansport        200000 non-null  bool   
 8   Brand_Nike            200000 non-null  bool   
 9   Brand_Puma            200000 non-null  bool   
 10  Brand_Under Armour    200000 non-null  bool   
 11  Brand_unknown         200000 non-null  bool   
 12  Material_Leather      200000 non-null  bool   
 13  Material_Nylon        200000 non-null  bool   
 14  Material_Polyester    200000 non-null  bool   
 15  

In [26]:
df_test.drop(['Unnamed: 10', 'Unnamed: 11'], axis=1, inplace=True)


In [27]:
df_test=pd.read_csv("Backpack_test1.csv")

In [28]:
df_test.head()

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Unnamed: 10,Unnamed: 11
0,300000,Puma,Leather,Small,2,0,0,Tote,Green,21,NaN,NaN
1,300001,Nike,Canvas,Medium,7,0,1,Backpack,Green,14,NaN,NaN
2,300002,Adidas,Canvas,Large,9,0,1,Messenger,Blue,12,NaN,NaN
3,300003,Adidas,Nylon,Large,1,1,0,Messenger,Green,18,NaN,NaN
4,300004,unknown,Nylon,Large,2,1,1,Tote,Black,10,NaN,NaN


In [29]:
df_test.drop(['Unnamed: 10', 'Unnamed: 11'], axis=1, inplace=True)

In [30]:
df_test['Price_per_capacity']=df['Price_per_capacity']

In [31]:
df_test = pd.get_dummies(df_test, columns=['Brand', 'Material', 'Style','Size','Color'], drop_first=True)

In [32]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 23 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    200000 non-null  int64  
 1   Compartments          200000 non-null  int64  
 2   Laptop Compartment    200000 non-null  int64  
 3   Waterproof            200000 non-null  int64  
 4   Weight Capacity (kg)  200000 non-null  int64  
 5   Price_per_capacity    200000 non-null  float64
 6   Brand_Jansport        200000 non-null  bool   
 7   Brand_Nike            200000 non-null  bool   
 8   Brand_Puma            200000 non-null  bool   
 9   Brand_Under Armour    200000 non-null  bool   
 10  Brand_unknown         200000 non-null  bool   
 11  Material_Leather      200000 non-null  bool   
 12  Material_Nylon        200000 non-null  bool   
 13  Material_Polyester    200000 non-null  bool   
 14  Style_Messenger       200000 non-null  bool   
 15  

In [33]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 23 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    300000 non-null  int64  
 1   Compartments          300000 non-null  int64  
 2   Laptop Compartment    300000 non-null  int64  
 3   Waterproof            300000 non-null  int64  
 4   Weight Capacity (kg)  300000 non-null  int64  
 5   Brand_Jansport        300000 non-null  bool   
 6   Brand_Nike            300000 non-null  bool   
 7   Brand_Puma            300000 non-null  bool   
 8   Brand_Under Armour    300000 non-null  bool   
 9   Brand_unknown         300000 non-null  bool   
 10  Material_Leather      300000 non-null  bool   
 11  Material_Nylon        300000 non-null  bool   
 12  Material_Polyester    300000 non-null  bool   
 13  Style_Messenger       300000 non-null  bool   
 14  Style_Tote            300000 non-null  bool   
 15  

In [36]:
common_columns = list(set(X.columns) & set(df_test.columns))
df_test = df_test[common_columns]  
df_test = df_test[X.columns]  

In [37]:
final_predictions =model.predict(df_test)

In [38]:
submission = pd.DataFrame({
    'id': df_test['id'],
    'Price': final_predictions
})

submission.to_csv("submission2.csv", index=False)
print("Submission file saved!")

Submission file saved!


In [39]:
import joblib

# Save the trained model
joblib.dump(model, "xgboost_model.pkl")


['xgboost_model.pkl']